In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/oneroom/items/{item_id}"

referer = "https://zigbang.com/home/oneroom/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/oneroom/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}
sad_words = [ '대출x', '대출불가', '전세 안됩']

In [13]:
def export_items(items):
    fieldnames = [
        'id',
        'title',
        'description',
        'deposit',
        'agent_name',
        'agent_mobile',
        '_floor',
        'address1',
        'address2',
        'address3',
        'size',
        'size_m2',
        'status',
        'room_type',
        'manage_cost_inc',
        'updated_at2',
        'url',
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=4)
    
    wish = '대출'
    for item in items:
        item = item['item']
        if wish in item['title'] or wish in item['description']:
            plz_flag = True
            for word in sad_words:
                if word in item['description']:
                    plz_flag = False
                    break
            if not plz_flag:
                continue
            item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
            if maximum_updated < datetime.datetime.now() - item_updated:
                continue
            item_id = item['id']
            url = magic_fstring_function(item_view_url)
            item['url'] = str(url)
            new_item = {}
            for fieldname in fieldnames:
                new_item[fieldname] = item[fieldname]
            filtered_item.append(new_item)
    return filtered_item
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/oneroom/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items
def get_room_list(
    items,
    deposit_gteq: int = 0,
    deposit_lteq: int = 8000,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wydjr",
    rent_gteq: int = 0,
    sales_type_in: str = "월세",
    service_type_eq: str = "원룸",
):
    resp = requests.get(
        url=item_list_api,
        params={
            "deposit_gteq": deposit_gteq,
            "deposit_lteq": deposit_lteq,
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
            "sales_type_in": sales_type_in,
            "service_type_eq": service_type_eq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [14]:
items = {}
geos_sinrim = [
    'wydm0',
    'wydm1',
    'wydm2',
   'wydm3',
] 

for geo in geos_sinrim:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [15]:
len(items)

2128

In [17]:
result = describe_room_list(items)
len(result)
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '3층',
  'address1': '서울시 관악구 봉천동',
  'address2': '41-670',
  'address3': None,
  'agent_address1': '서울특별시 관악구 관악로16길 42 1층(봉천동)',
  'agent_comment': '',
  'agent_email': 'cjsrlgh1@gmail.com',
  'agent_lat': 37.479643300561705,
  'agent_lng': 126.954969531442,
  'agent_local1': '서울시',
  'agent_local2': '관악구',
  'agent_mobile': '010-9596-0920',
  'agent_name': '구름공인중개사사무소(천기호)',
  'agent_no': 71941,
  'agent_phone': '010-9596-0920',
  'bjd_code': '',
  'bonbun_code': '41',
  'bubun_code': '670',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 1000,
  'description': '💖💖매물로 승부봅니다.💖💖\r\n'
                 '✨더 좋은방, 더 싸게✨\r\n'
                 '\r\n'
                 '✅ 서울대입구역 7분\r\n'
                 '✅ 방1, 욕실1\r\n'
                 '✅ 현관보안, 에어컨, 세탁기, 냉장고, 옷장 등 풀옵션!!\r\n'
                 '\r\n'
                 '≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡\r\n'
                 '\r\n'
                 '✨방 구조✨\r\n'
                 '✔ 오픈형 원룸(방1, 욕실1,)\r\n'
           

In [8]:
filtered_result = export_items(result)
len(filtered_result)

26

In [10]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr
geo_based_dict = {}
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)
# 원하는 조건 가진 아이템을 csv 형식으로 파일을 만든다.

with open('eggs.csv', 'w', newline='') as csvfile:
    fieldnames = [
        'id',
        'title',
        'description',
        'deposit',
        'agent_name',
        'agent_mobile',
        '_floor',
        'address',
        'size',
        'size_m2',
        'status',
        'room_type',
        'manage_cost_inc',
        'updated_at2',
        'url',
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

{ '_floor': '4층',
  'address': '서울시 관악구 신림동 99-39 ',
  'address1': '서울시 관악구 신림동',
  'address2': '99-39',
  'address3': None,
  'agent_mobile': '010-4711-7462',
  'agent_name': '동그라미공인중개사사무소(정승우)',
  'deposit': 5000,
  'description': '서림동 서원역(5월개통) 도보 7분거리에 \r\n'
                 '전세자금대출 가능한  집이 나왔습니다^^\r\n'
                 '\r\n'
                 '⚘금액⚘\r\n'
                 '✔전세금 5000만원 [전세자금대출가능^^]\r\n'
                 '✔관리비 5만원(인터넷, TV,수도,정화비)포함, 전기,가스별도^^ \r\n'
                 '\r\n'
                 '⚘ 방구조 ⚘ \r\n'
                 '✔ 오픈형 구조로 공간활용을 넓게 사용가능해요 ^^ \r\n'
                 '✔ 창문이 커서 채광과 통풍이 잘되요 선글라스필수^^ \r\n'
                 '\r\n'
                 '⚘ 주변환경 및 교통편 ⚘ \r\n'
                 '✔ 서원역 도보7분거리에 위치해 있습니다^^  \r\n'
                 '✔ 버스 정류장옆이라 교통이 뛰어나고 편리해요^^ \r\n'
                 '✔ 대로변 인근이라 치안이 좋고 cctv 및 보안이 짱짱해요 \r\n'
                 '✔ 주변에 대형마트, 편의점, 맛집등 편의시설이 좋아요\r\n'
                 '\r\n'
                 '⚘ 옵션 ⚘ \r\n'
                 '✔ tv, 옷장, 인덕션, 냉

UnicodeEncodeError: 'cp949' codec can't encode character '\U0001f525' in position 9: illegal multibyte sequence